In [2]:
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

In [9]:
DATA_URL = 'https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.zip'
DATA_TEMPLATE_URL = 'https://raw.githubusercontent.com/DataScienceResearchPeru/covid-19_latinoamerica/master/latam_covid_19_data/templates/daily_report.csv'
PATH_CSV = "argentina_temporal/"
PATH_DSRP_DAILY_REPORTS = 'latam_covid_19_data/daily_reports/'
LAST_UPDATE = datetime.today().isoformat()
DICT_PLACES = {'Buenos Aires':'AR-B',
               'CABA':'AR-C',
               'Catamarca':'AR-K',
               'Chaco':'AR-H',
               'Chubut':'AR-U',
               'Corrientes':'AR-W',
               'Córdoba':'AR-X',
               'Entre Ríos':'AR-E',
               'Formosa':'AR-P',
               'Indeterminado':'',
               'Jujuy':'AR-Y',
               'La Pampa':'AR-L',
               'La Rioja': 'AR-F',
               'Mendoza':'AR-M',
               'Misiones': 'AR-N',
               'Neuquén':'AR-Q',
               'Río Negro':'AR-R',
               'Salta':'AR-A',
               'San Juan':'AR-J',
               'San Luis':'AR-D',
               'Santa Cruz':'AR-Z',
               'Santa Fe':'AR-S',
               'Santiago del Estero':'AR-G',
               'Tierra del Fuego':'AR-V',
               'Tucumán':'AR-T'
              }

In [10]:
import requests
url = DATA_URL
headers = {"user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}

PATH_FILE=PATH_CSV+"data.zip"
print(PATH_FILE)

if not os.path.isfile(PATH_FILE):
    r = requests.get(url, headers=headers)

    with open(PATH_FILE,'wb') as f:
        f.write(r.content)
else:
    print('[ARGENTINA] FILE ZIP ALREADY EXISTS, USING THAT FILE')


df = pd.read_csv(PATH_FILE, 
                 usecols=['residencia_provincia_nombre','fecha_diagnostico','ultima_actualizacion','clasificacion_resumen','fallecido'],
                 compression="zip")

argentina_temporal/data.zip
[ARGENTINA] FILE ZIP ALREADY EXISTS, USING THAT FILE


In [11]:
df.head()
#parse_dates=['fecha_diagnostico','ultima_actualizacion'],

,residencia_provincia_nombre,fallecido,clasificacion_resumen,fecha_diagnostico,ultima_actualizacion
0,CABA,NO,Descartado,2020-06-09,2021-05-27
1,CABA,NO,Descartado,2021-04-08,2021-05-27
2,Buenos Aires,NO,Descartado,2021-04-08,2021-05-27
3,Buenos Aires,NO,Descartado,2021-04-06,2021-05-27
4,CABA,NO,Descartado,2021-04-08,2021-05-27


In [12]:
df_confirmed=df[df['clasificacion_resumen']=='Confirmado'].groupby(['residencia_provincia_nombre','fecha_diagnostico']).count() \
  .groupby(level=0).cumsum().reset_index()

df_confirmed['ISO 3166-2 Code']=df_confirmed['residencia_provincia_nombre'].map(DICT_PLACES)
df_confirmed=df_confirmed[['ISO 3166-2 Code','fecha_diagnostico','ultima_actualizacion']]
df_confirmed=df_confirmed.dropna()

df_confirmed[(df_confirmed['fecha_diagnostico']=='2021-05-22')&(df_confirmed['ISO 3166-2 Code']=='AR-B')]['ultima_actualizacion'].values[0]

1350182

In [28]:
df_deaths=df[df['fallecido']=='SI'].groupby(['residencia_provincia_nombre','fecha_diagnostico']).count() \
  .groupby(level=0).cumsum().reset_index()

df_deaths['ISO 3166-2 Code']=df_deaths['residencia_provincia_nombre'].map(DICT_PLACES)
df_deaths=df_deaths[['ISO 3166-2 Code','fecha_diagnostico','ultima_actualizacion']]
df_deaths=df_deaths.dropna()

df_deaths[df_deaths['fecha_diagnostico']=='2021-05-22']

,ISO 3166-2 Code,fecha_diagnostico,ultima_actualizacion
431,AR-B,2021-05-22,44777
866,AR-C,2021-05-22,10744
1358,AR-H,2021-05-22,1458
2513,AR-E,2021-05-22,1587
4751,AR-A,2021-05-22,1447
4945,AR-J,2021-05-22,541
5970,AR-G,2021-05-22,484


In [21]:
df_template=pd.read_csv(DATA_TEMPLATE_URL)
df_template=df_template.fillna(0)
#df_template['Confirmed']=df_template['Confirmed'].astype(int)
#df_template['Deaths']=df_template['Deaths'].astype(int)
#df_template['Recovered']=df_template['Recovered'].astype(int)
df_template['Last Update']=LAST_UPDATE
df_template.head()

,ISO 3166-2 Code,Country,Subdivision,Last Update,Confirmed,Deaths,Recovered
0,AR-B,Argentina,Buenos Aires,2021-05-23T23:10:17.638583,0.0,0.0,0.0
1,AR-K,Argentina,Catamarca,2021-05-23T23:10:17.638583,0.0,0.0,0.0
2,AR-H,Argentina,Chaco,2021-05-23T23:10:17.638583,0.0,0.0,0.0
3,AR-U,Argentina,Chubut,2021-05-23T23:10:17.638583,0.0,0.0,0.0
4,AR-C,Argentina,Ciudad Autonoma de Buenos Aires,2021-05-23T23:10:17.638583,0.0,0.0,0.0


In [ ]:
day='2021-05-01'
for country_code in df_confirmed['ISO 3166-2 Code']:
    print(day,country_code)
    day_adapted=datetime.strptime(day, '%Y-%m-%d').strftime('%d/%m/%Y')
    value=df_confirmed[df_confirmed['ISO 3166-2 Code']==country_code][day_adapted].values[0]
    df_template.loc[df_template['ISO 3166-2 Code']==country_code,'Confirmed']=int(value)
    
    
    day_adapted=datetime.strptime(day, '%Y-%m-%d').strftime('%d/%m/%Y')
    value=df_confirmed[df_confirmed['ISO 3166-2 Code']==country_code][day_adapted].values[0]
    df_template.loc[df_template['ISO 3166-2 Code']==country_code,'Deaths']=int(value)

In [38]:
for a in df_confirmed['ISO 3166-2 Code'].unique():
    print(a)

AR-B
AR-C
AR-K
AR-H
AR-U
AR-W
AR-X
AR-E
AR-P
AR-Y
AR-L
AR-F
AR-M
AR-N
AR-Q
AR-R
AR-A
AR-J
AR-D
AR-Z
AR-S
AR-G
AR-V
AR-T
